
# Jupyter Notebooks for the book: Semiconductor Integrated Optics for Switching Light 2nd Edition
#Notebook for Chapter 2:
 
## Franz-Keldysh effect, electroabsorption modulator, and electrorefraction in $In_{1-x-y}Ga_xAl_yAs$ 

Prof. Charlie Ironside,
Department of Physics and Astronomy,
Curtin University,
Bentley Campus,
Western Australia 6102.

* email: Charlie.Ironside@curtin.edu.au

* web page address:http://oasisapps.curtin.edu.au/staff/profile/view/Charlie.Ironside


This program plots the wavelength dependence of the Franz-Kelydysh effect and electrofraction in the semiconductor alloy $In_{1-x-y}Ga_xAl_yAs$

First the Franz-Keldydh effect is calculated that is the change in the absorption coefficient induced by an electric field in the semiconductor alloy. The F1 electric field models the field present in a pn junction (or due to band-edge smearing from alloy or poor growth effects) before the externally applied field F2 is applied. Both electric fields are in kV/cm

The electrorefraction effect is the change refractive index induced in the semiconductor by an electric field. It's calculated by taking the Kramers-Kronig transformation of the Franz-Keldysh effect. The Kramers-Kronig transformation is worked out via a Hilbert transform. Before the Hilbert transform can be used for a Kramers-Krong transform the change in absorption coefficient needs to be recast as change in extinction coefficient.

This program models alloys have the general formulae $In_{1-x-y}Ga_xAl_yAs$ Once y is selected x is determined by lattice matching considerations, the parameter used below is z=y/0.48. For latticed matched alloy, y, can only range between 0 and 0.48. y is an adjustable paramenter. y=0.06 sets the band-gap energy to close the photon energy for photons with a wavelength of 1500nm.

## References

1.   D. C. Hutchings, M. Sheik-Bahae, D. J. Hagan and E. W. Van Stryland, 
"Kramers-Kronig relations in nonlinear optics" Optical and Quantum Electronics 24 1992 1-30 http://dx.doi.org/10.1007/Bf01234275
2.   A. Alping and L. A. Coldren, "Electrorefraction in GaAs and InGaAsP and Its Application to Phase Modulators," Journal of Applied Physics, vol. 61, pp. 2430-2433, Apr 1 1987.http://dx.doi.org/10.1063/1.337912
3. A. R. Forouhi and I. Bloomer, "Optical properties of crystalline semiconductors and dielectrics," Physical Review B, Article vol. 38, no. 3, pp. 1865-1874, Jul 1988. http://dx.doi.org/10.1103/PhysRevB.38.1865
4.Jose M. L. Figueiredo "Optoelectronic properties of resonant tunneling diodes", (P220), Chpt 6 PhD thesis,  2000 University of Porto. http://userweb.eng.gla.ac.uk/charles.ironside/JLFThesis.pdf





This part plots the absorption spectrum at 2 different electric fields, F1 and F2

In [1]:
# Franz-Kelydysh effect in InGaAlAs using Airy functions
# Airy functions from https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.airy.html
from scipy import special
import matplotlib.pyplot as plt
import numpy as np
import math
import ipywidgets as widgets
from matplotlib.transforms import Transform
from matplotlib.ticker import (
    AutoLocator, AutoMinorLocator)

def forward(x):
    return 1240 / x
def inverse(x):
    """
    this function converts eV to nm for axis plot
    """
    return 1240 /x 

def franz_keldysh_plot(plt,y,F1,F2):
    """
    this function plots the absorption coefficient versus electron volts of a semiconductor In1-xGaxAlyAs with
    y Al fraction at 2 different applied fields, F1 and F2
    for plt is plot range given as a fraction of the bandgap EG
    """
    hbar=1.055E-27 #erg-sec
    Echarge=4.80325E-10 # ESU
    Emass=9.109558E-28 #g
    z=y/0.48 # used in calculating bandgap
    EG= 0.76 +0.49*z + 0.20*z**2 # Calculates bandgap from y fraction of In1-x-yGaAlyAs in eV Jose M. L. Figueiredo "Optoelectronic properties of resonant tunneling diodes", (P220), PhD thesis,  2000 University of Porto.
    Cgap=1.76E12# *cm-1sec-1/2*);(*Actually for GaAs but we assume much the same for all III-V semiconductors - from Seraphin and Bottka Phys Rev A 139 560 1965*)
    me=0.0427 + 0.0328*z #electron mass
    mh=0.5 #hole mass
    mr=(me*mh)/(me+mh) #reduced mass
    mass=mr*Emass #mass used in Franz Keldysh calculation
    eV = np.linspace(EG-(plt*EG),EG+(plt*EG),256) # the electron volta linear scale
    theta1=((Echarge**2*(F1*10/3)**2)/(2*mass*hbar))**(1/3)# Background electrical field scaled for Franz-Kelydsh effect
    theta2=((Echarge**2*(F2*10/3)**2)/(2*mass*hbar))**(1/3)# Applied electrical field scaled for Franz-Kelydsh effect
    etta1=(EG-eV)/(hbar*theta1/1.60219E-12)# set the energy scale
    etta2=(EG-eV)/(hbar*theta2/1.60219E-12)
    ai1, aip1, bi1, bip1 = special.airy(etta1)#gets the Airy functions
    ai2, aip2, bi2, bip2 = special.airy(etta2)#gets the Airy functions
    alpha1=(Cgap*math.sqrt(theta1))/(eV *1.5193E15)*((aip1*aip1)-etta1*ai1*ai1)
    alpha2=(Cgap*math.sqrt(theta2))/(eV *1.5193E15)*((aip2*aip2)-etta2*ai2*ai2)
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1, figsize=(16, 8))
    ax.clear()
    units1 = 'y = {} $(Al)$ Field = {} $(kV/cm)$'
    units2 = 'y = {} $(Al)$ Field = {} $(kV/cm)$'
    plt.suptitle('Franz-Keldysh Effect  $In_{1-x-y} Ga_x Al_y As$',fontsize=20)
    plt.plot(eV,alpha1,'b-',label=units1.format(y,F1))
    plt.plot(eV,alpha2,'r-',label=units2.format(y,F2))
    plt.ylabel('Absorption coefficient 1/cm',fontsize=20)
    plt.grid()
    plt.legend(loc=2)
    plt.xlabel('eV',fontsize=20)
    secax = ax.secondary_xaxis('top',functions=(forward, inverse))#puts wavelength nm axis on top of plot
    secax.set_xlabel('Wavelength [nm]')#puts wavelength nm axis on top of plot
    plt.show()
    
""" This part deals with taking values from the sliders and calling the plotting functions"""
y= widgets.FloatSlider(min=0, max=0.5, value=0.05, description='Al fraction y:')
plt= widgets.FloatSlider(min=0.05, max=1, value=0.1, description='Plot range plt:')
F1= widgets.FloatSlider(min=0.005, max=10, value=5, description='Electric Field F1')
F2= widgets.FloatSlider(min=0.005, max=100, value=50, description='Electric Field F2')
widgets.interactive(franz_keldysh_plot, plt=plt, y=y, F1=F1, F2=F2)


interactive(children=(FloatSlider(value=0.1, description='Plot range plt:', max=1.0, min=0.05), FloatSlider(va…

This part plots the change in the absorption coefficient

In [ ]:
# Franz-Kelydysh effect in InGaAlAs using Airy functions
# Airy functions from https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.airy.html
from scipy import special
import matplotlib.pyplot as plt
import numpy as np
import math
import ipywidgets as widgets
from matplotlib.transforms import Transform
from matplotlib.ticker import (
    AutoLocator, AutoMinorLocator)

def forward(x):
    return 1240 / x
def inverse(x):
    """
    this function converts eV to nm for axis plot
    """
    return 1240 /x 

def franz_keldysh_plot(plt,y,F1,F2):
    """
    this function plots the difference in absorption coefficient versus electron volts of a semiconductor In1-xGaxAlyAs with
    y Al fraction
    with a background field F1 (V/cm)
    and applied field F2 (V/cm)
    for plt is plot range given as a fraction of the bandgap EG
    """
    hbar=1.055E-27 #erg-sec
    Echarge=4.80325E-10 # ESU
    Emass=9.109558E-28 #g
    z=y/0.48 # used in calculating bandgap
    EG= 0.76 +0.49*z + 0.20*z**2 # Calculates bandgap from y fraction of In1-x-yGaAlyAs in eV Jose M. L. Figueiredo "Optoelectronic properties of resonant tunneling diodes", (P220), PhD thesis,  2000 University of Porto.
    Cgap=1.76E12# *cm-1sec-1/2*);(*Actually for GaAs but we assume much the same for all III-V semiconductors - from Seraphin and Bottka Phys Rev A 139 560 1965*)
    me=0.0427 + 0.0328*z #electron mass
    mh=0.5 #hole mass
    mr=(me*mh)/(me+mh) #reduced mass
    mass=mr*Emass #mass used in Franz Keldysh calculation
    eV = np.linspace(EG-(plt*EG),EG+(plt*EG),256) # the electron volta linear scale
    theta1=((Echarge**2*(F1*10/3)**2)/(2*mass*hbar))**(1/3)# Background electrical field scaled for Franz-Kelydsh effect
    theta2=((Echarge**2*(F2*10/3)**2)/(2*mass*hbar))**(1/3)# Applied electrical field scaled for Franz-Kelydsh effect
    etta1=(EG-eV)/(hbar*theta1/1.60219E-12)# set the energy scale
    etta2=(EG-eV)/(hbar*theta2/1.60219E-12)
    ai1, aip1, bi1, bip1 = special.airy(etta1)#gets the Airy functions
    ai2, aip2, bi2, bip2 = special.airy(etta2)#gets the Airy functions
    alpha1=(Cgap*math.sqrt(theta1))/(eV *1.5193E15)*((aip1*aip1)-etta1*ai1*ai1)
    alpha2=(Cgap*math.sqrt(theta2))/(eV *1.5193E15)*((aip2*aip2)-etta2*ai2*ai2)
    delta=alpha2-alpha1
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1, figsize=(16, 8))
    ax.clear()
    units = 'y = {} $(Al)$ F1 = {} $(kV/cm)$ F2 = {} $(kV/cm)$'
    plt.suptitle('Franz-Keldysh Effect  $In_{1-x-y} Ga_x Al_y As$',fontsize=20)
    plt.plot(eV,delta,'b-',label=units.format(y,F1,F2))
    plt.ylabel('$\Delta$ absorption coefficient 1/cm',fontsize=20)
    plt.grid()
    plt.legend(loc=2)
    plt.xlabel('eV',fontsize=20)
    secax = ax.secondary_xaxis('top',functions=(forward, inverse))#puts wavelength nm axis on top of plot
    secax.set_xlabel('Wavelength [nm]')#puts wavelength nm axis on top of plot
    plt.show()
    
""" This part deals with taking values from the sliders and calling the plotting functions"""
y= widgets.FloatSlider(min=0, max=0.5, value=0.05, description='Al fraction y:')
plt= widgets.FloatSlider(min=0.05, max=1, value=0.1, description='Plot range plt:')
F1= widgets.FloatSlider(min=0.005, max=10, value=5, description='B field F1')
F2= widgets.FloatSlider(min=0.005, max=100, value=50, description='Applied F2')
widgets.interactive(franz_keldysh_plot, plt=plt, y=y, F1=F1, F2=F2)


interactive(children=(FloatSlider(value=0.1, description='Plot range plt:', max=1.0, min=0.05), FloatSlider(va…


## Franz-Keldysh Modulator - Figure of merit- $In_{1-x-y}Ga_xAl_yAs$ 



This sectioninteractively plots the wavelength dependence of the figure of merit for a electroabsorption modulator operating at around optical telecoms wavelengths using s bulk alloy semiconductor for a given applied electric field via the Franz-Keldysh The F1 field models the field present in a pn junction (or due to band-edge smearing from alloy or poor growth effects) before the externally applied field F2 is applied. Both fields are in kV/cm

This program models alloys have the general $In_{1-x-y}Ga_xAl_yAs$ Once y is selected x is determined by lattice matching considerations, the parameter used below is z=y/0.48. For latticed matched alloy, y, can only range between 0 and 0.48. y is an adjustable paramenter. y=0.05 sets the band-gap energy to close the photon energy for photons with a wavelength of 1500nm.

## References

1.   D. C. Hutchings, M. Sheik-Bahae, D. J. Hagan and E. W. Van Stryland, 
"Kramers-Kronig relations in nonlinear optics" Optical and Quantum Electronics 24 1992 1-30 http://dx.doi.org/10.1007/Bf01234275
2.   A. Alping and L. A. Coldren, "Electrorefraction in GaAs and InGaAsP and Its Application to Phase Modulators," Journal of Applied Physics, vol. 61, pp. 2430-2433, Apr 1 1987.http://dx.doi.org/10.1063/1.337912
3. M. K. Chin, "On the figures of merit for electroabsorption wave-guide modulators," IEEE Photonics Technology Letters, vol. 4, pp. 726-728, Jul 1992 http://dx.doi.org/10.1109/68.145252


In [ ]:
# Airy functions from https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.airy.html
#see Chin reference - figure of merit for an optical modualtor
from scipy import special
import matplotlib.pyplot as plt
import numpy as np
import math
import ipywidgets as widgets

def forward(x):
    return 1240 / x
def inverse(x):
    """
    this function converts eV to nm for axis plot
    """
    return 1240 /x 
def F_K_Mod_plot(plt,y,F1,F2):
    """
    this is implements the merit of merit for a Franz-Keldysh effect based electroabsorption modualtor - see Chin ref
    this function plots the change in absorption coefficient /Absorption coefficient versus electron volts of a semiconductor In1-xGaxAlyAs with
    y Al fraction
    with a background field F1 (V/cm)
    and applied field F2 (V/cm)
    for plt is plot range given as a fraction of the bandgap EG
    """
    hbar=1.055E-27 #erg-sec
    Echarge=4.80325E-10 # ESU
    Emass=9.109558E-28 #g
    z=y/0.48 # used in calculating bandgap
    EG= 0.76 +0.49*z + 0.20*z**2 # Calculates bandgap from y fraction of In1-x-yGaAlyAs in eV
    Cgap=1.76E12# *cm-1sec-1/2*);(*Actually for GaAs but we assume much the same for all III-V semiconductors - from Seraphin and Bottka Phys Rev A 139 560 1965*)
    me=0.0427 + 0.0328*z #electron mass
    mh=0.5 #hole mass
    mr=(me*mh)/(me+mh) #reduced mass
    mass=mr*Emass #mass used in Franz Keldysh calculation
    eV = np.linspace(EG-(plt*EG),EG+((plt/2)*EG),256) # the electron volta linear scale
    theta1=((Echarge**2*(F1*10/3)**2)/(2*mass*hbar))**(1/3)# Background electrical field scaled for Franz-Kelydsh effect
    theta2=((Echarge**2*(F2*10/3)**2)/(2*mass*hbar))**(1/3)# Applied electrical field scaled for Franz-Kelydsh effect
    etta1=(EG-eV)/(hbar*theta1/1.60219E-12)# set the energy scale
    etta2=(EG-eV)/(hbar*theta2/1.60219E-12)
    ai1, aip1, bi1, bip1 = special.airy(etta1)#gets the Airy functions
    ai2, aip2, bi2, bip2 = special.airy(etta2)
    alpha1=(Cgap*math.sqrt(theta1))/(eV *1.5193E15)*((aip1*aip1)-etta1*ai1*ai1)
    alpha2=(Cgap*math.sqrt(theta2))/(eV *1.5193E15)*((aip2*aip2)-etta2*ai2*ai2)
    balpha=10 #scattering loss in waveguide absorption in cm-1
    deltaopt=(alpha2-alpha1)/(alpha1+balpha)
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1, figsize=(16, 8))
    ax.clear()
    units = 'y = {} $(Al)$ F1 = {} $(kV/cm)$ F2 = {} $(kV/cm)$'
    plt.suptitle('Figure of merit for electroabsorption in $In_{1-x-y} Ga_x Al_y As$',fontsize=24)
    plt.plot(eV,deltaopt,'b-',label=units.format(y,F1,F2))
    plt.ylabel('$\Delta$ $\\alpha/\\alpha$',fontsize=20)
    plt.grid()
    plt.legend(loc=2)
    plt.xlabel('eV',fontsize=20)
    secax = ax.secondary_xaxis('top',functions=(forward, inverse))#puts wavelength nm axis on top of plot
    secax.set_xlabel('Wavelength [nm]',fontsize=14)#puts wavelength nm axis on top of plot
    plt.show()
""" This part deals with taking values from the sliders and calling the plotting functions"""
y= widgets.FloatSlider(min=0, max=0.48, value=0.06, description='Al fraction y:')
plt= widgets.FloatSlider(min=0.05, max=1, value=0.1, description='Plot range plt:')
F1= widgets.FloatSlider(min=0.005, max=10, value=5, description='B field F1')
F2= widgets.FloatSlider(min=0.005, max=200, value=50, description='Applied F2')
widgets.interactive(F_K_Mod_plot, plt=plt, y=y, F1=F1, F2=F2)

interactive(children=(FloatSlider(value=0.1, description='Plot range plt:', max=1.0, min=0.05), FloatSlider(va…

# Electrorefraction

In this section electrofraction spectrum associated with the Franz-Feldysh effect spectrum is calculated using the Hilbert transform method reported by Whittaker et al. 


## References

B. O. Seraphin and N. Bottka, "Franz-Keldysh Effect of the Refractive Index in Semiconductors," Physical Review, vol. 139, no. 2A, pp. A560-A565, 07/19/ 1965.

A. Alping and L. A. Coldren, "Electrorefraction in GaAs and InGaAsP and Its Application to Phase Modulators," Journal of Applied Physics, vol. 61, pp. 2430-2433, Apr 1 1987.http://dx.doi.org/10.1063/1.337912

K. A. Whittaker, J. Keaveney, I. G. Hughes, and C. S. Adams, "Hilbert transform: Applications to atomic spectra," Physical Review A, vol. 91, no. 3, Mar 25 2015.

See also
http://people.exeter.ac.uk/sh481/paradoxes.html

 http://people.exeter.ac.uk/sh481/kramers-kronig-relations.html




In [ ]:
from scipy import special
import matplotlib.pyplot as plt
import numpy as np
import math
import ipywidgets as widgets
import scipy.fftpack as ft # Contains a function that will compute the Hilbert transform
from matplotlib.transforms import Transform
from matplotlib.ticker import (
    AutoLocator, AutoMinorLocator)
def forward(x):
    return 1240 / x
def inverse(x):
    return 1240 /x 
def Erefract_plot(plt,y,F1,F2):
  hbar=1.055E-27 #erg-sec
  Echarge=4.80325E-10 # ESU
  c=2.9979E10 #Speed of light in cm/s
  eVtoHz= 2.417989E14 #multiple by this factor to convert ev to Hz
  pi=3.1415#value of pi
  Emass=9.109558E-28 #g
  z=y/0.48 # used in calculating bandgap
  EG= 0.76 +0.49*z + 0.20*z**2 # Calculates bandgap from y fraction of In1-x-yGaAlyAs in eV
  Cgap=1.76E12# *cm-1sec-1/2*);(*Actually for GaAs but we assume much the same for all III-V semiconductors - from Seraphin and Bottka Phys Rev A 139 560 1965*)
  me=0.0427 + 0.0328*z #electron mass
  mh=0.5 #hole mass
  mr=(me*mh)/(me+mh) #reduced massmass=mr*Emass #mass used in Franz Keldysh calculation
  mass=mr*Emass #mass used in Franz Keldysh calculation
  eV = np.linspace(0.01,4*EG,4000) # the electron volts linear scale - over the range 0-4 *EG (bandgap energy) wide range required for accurate values of change in refractive index
  theta1=((Echarge**2*(F1*10/3)**2)/(2*mass*hbar))**(1/3)# Background electrical field scaled for Franz-Kelydsh effect
  theta2=((Echarge**2*(F2*10/3)**2)/(2*mass*hbar))**(1/3)# Applied electrical field scaled for Franz-Kelydsh effect
  etta1=(EG-eV)/(hbar*theta1/1.60219E-12)# set the energy scale
  etta2=(EG-eV)/(hbar*theta2/1.60219E-12)
  ai1, aip1, bi1, bip1 = special.airy(etta1)#gets the Airy functions
  ai2, aip2, bi2, bip2 = special.airy(etta2)#gets the Airy functions
  alpha1=(Cgap*math.sqrt(theta1))/(eV *1.5193E15)*((aip1*aip1)-etta1*ai1*ai1)
  alpha2=(Cgap*math.sqrt(theta2))/(eV *1.5193E15)*((aip2*aip2)-etta2*ai2*ai2)
  delta_alpha=(alpha2-alpha1)
  ext_coeff=delta_alpha*eV/(2*pi*eVtoHz/c)# see Whittaker et al. converts absorption coefficient to extinction coefficient,k, - (2*pi*eVtoHz/c) is the conversion factor for k 
  efract=ft.hilbert(ext_coeff)
  range1=int(len(eV)/4-plt*len(eV)/2) # only plot narrow range of data
  range2=int(len(eV)/4+plt*len(eV)/2) # only plot narrow range of data
  pefract=efract[range1:range2]# only plot narrow range of data
  peV=eV[range1:range2]# only plot narrow range of data
  import matplotlib.pyplot as plt
  fig, ax = plt.subplots(1, figsize=(16, 8))
  ax.clear()
  units = 'y = {} $(Al)$ F1 = {} $(kV/cm)$ F2 = {} $(kV/cm)$'
  plt.suptitle('Electrorefraction Effect in $In_{1-x-y} Ga_x Al_y As$',fontsize=20)
  plt.plot(peV,pefract,'r-',lw=1,label=units.format(y,F1,F2))
  #plt.plot(eV,delta,'b-',lw=1,label="$\Delta$ $\\alpha$ Absorption Coeff")
  #plt.plot(eV,alpha2,'g-',lw=1,label="absorption coeff")
  plt.ylabel('$\Delta$ n',fontsize=20)
  plt.grid()
  plt.xlabel('eV',fontsize=20)
  plt.legend()
  secax = ax.secondary_xaxis('top',functions=(forward, inverse))
  secax.set_xlabel('Wavelength [nm]')
  plt.show()
  
""" This part deals with taking values from the sliders and calling the plotting functions"""
y= widgets.FloatSlider(min=0, max=0.48, value=0.05, description='Al fraction y:')
plt= widgets.FloatSlider(min=0.01, max=0.25, value=0.1, description='Plot range plt:')
F1= widgets.FloatSlider(min=0.005, max=10, value=5, description='B field F1')
F2= widgets.FloatSlider(min=0.005, max=500, value=50, description='Applied F2')
widgets.interactive(Erefract_plot, plt=plt, y=y, F1=F1, F2=F2)

interactive(children=(FloatSlider(value=0.1, description='Plot range plt:', max=0.25, min=0.01), FloatSlider(v…